<a href="https://colab.research.google.com/github/NINGTANG1124/UPF-HFI/blob/main/notebooks/step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# connect googledrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read intake data (including Descriptionen and FoodGroupen)
import pandas as pd
import re

file_path = "/content/drive/MyDrive/UPF-HFI/Bradford_original data/1. Dietmasterfile_foodlevel_clean.xls"
intake_df = pd.read_excel(file_path)

# Define text cleaning function
def clean_text(col):
    return col.astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

# Apply to key fields
intake_df["Foodgroupen_clean"] = clean_text(intake_df["Foodgroupen"])
intake_df["Descriptionen_clean"] = clean_text(intake_df["Descriptionen"])

# 添加 Subgroupcode 的清洗列
intake_df["Subgroupcode_clean"] = clean_text(intake_df["Subgroupcode"])


In [4]:
# 设置 att3 文件路径
att3_path = "/content/drive/MyDrive/UPF-HFI/nova matching files/att3-excel.xlsx"

# 读取 att3 文件
att3 = pd.read_excel(att3_path)

# 清洗 group code（假设列名为 'Subsidiary food group code'）
att3['code_clean'] = att3['Subsidiary food group code'].astype(str).str.upper().str.strip()

# 创建字典（包括 *）
group_to_nova = dict(zip(att3['code_clean'], att3['NOVA food group']))

# 清洗 intake 数据的 Subgroupcode 列
intake_df['Subgroupcode_clean'] = intake_df['Subgroupcode'].astype(str).str.upper().str.strip()


In [ ]:
# Step 1: 映射 group code 到 NOVA_step1
intake_df['NOVA_step1'] = intake_df['Subgroupcode_clean'].map(group_to_nova)

# Step 2: 添加匹配说明列（仅记录成功匹配的）
intake_df['match_reason_step1'] = intake_df['NOVA_step1'].apply(
    lambda x: 'group code match' if str(x).isdigit() else None
)

# 哪些行已经匹配成功？多少还没有？
# 匹配情况预览
intake_df['NOVA_step1'].value_counts(dropna=False)


In [ ]:
# 导出检查文件（只保留关键列）
cols_to_export = [
    'Descriptionen', 'Foodgroupen',
    'Subgroupcode', 'Subgroupcode_clean',
    'NOVA_step1', 'match_reason_step1'
]

check_df = intake_df[cols_to_export].copy()
check_df.to_excel("/content/drive/MyDrive/UPF-HFI/0728outcome/step1.xlsx", index=False)


# 精准提取 Step 2 目标项（NOVA_step1 是 * 或 **）

In [ ]:
# Step 2：提取目标记录（NOVA 是 * 或 **）
df_step2_targets = intake_df[intake_df['NOVA_step1'].isin(["*", "**"])].copy()

# 按 Subgroupcode_clean 去重，只保留每组一个代表行
df_step2_unique = df_step2_targets.drop_duplicates(subset="Subgroupcode_clean")

# 选择导出的关键列
cols_step2 = [
    'Descriptionen', 'Foodgroupen',
    'Subgroupcode', 'Subgroupcode_clean',
    'NOVA_step1', 'match_reason_step1'
]
df_step2_unique_simple = df_step2_unique[cols_step2]

# 导出为 Excel 文件
df_step2_unique_simple.to_excel("/content/drive/MyDrive/UPF-HFI/0728outcome/step2_groupcodes_unique.xlsx", index=False)


# 提取NaN


In [1]:
# 提取 NOVA_step1 为 NaN 的记录（不去重）
df_step2_nan_all = intake_df[intake_df['NOVA_step1'].isna()].copy()

# 选择导出的关键列
cols_step2_nan = [
    'Descriptionen', 'Foodgroupen',
    'Subgroupcode', 'Subgroupcode_clean',
    'NOVA_step1', 'match_reason_step1'
]
df_step2_nan_all_simple = df_step2_nan_all[cols_step2_nan]

# 导出为 Excel 文件
df_step2_nan_all_simple.to_excel(
    "/content/drive/MyDrive/UPF-HFI/0728outcome/step2_groupcodes_NaN_all.xlsx",
    index=False
)


NameError: name 'intake_df' is not defined